In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
import torch
from torch import Tensor

import lincoln
from lincoln.layers import Dense
from lincoln.losses import LogSoftmaxLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork

## Regression network

In [4]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

## Regression models

In [5]:
from lincoln.losses import MeanSquaredError
from lincoln.optimizers import SGD

from lincoln.operations.activations import Sigmoid, LinearAct

#### Linear Regression

In [6]:
from lincoln.train import Trainer

In [7]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer = Trainer(lr, SGD())

In [8]:
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 6054.131
Validation loss after 2 epochs is 4165.070
Validation loss after 3 epochs is 5354.270
Validation loss after 4 epochs is 4191.684
Validation loss after 5 epochs is 3958.340
Validation loss after 6 epochs is 4461.651
Validation loss after 7 epochs is 3746.806
Validation loss after 8 epochs is 4463.007
Validation loss after 9 epochs is 4598.510
Validation loss after 10 epochs is 3916.603


In [9]:
def mae(y_true, y_pred):
    return round(torch.mean(torch.abs(y_true - y_pred)).item(), 4)

def mse(y_true, y_pred):
    return round(torch.mean(torch.pow(y_true - y_pred, 2)).item(), 4)

In [10]:
def eval_regression_model(lr: NeuralNetwork, 
                          X_test: Tensor, 
                          y_test: Tensor):
    preds = lr.forward(X_test)
    preds = preds.reshape(preds.shape[0])
    return mae(preds, y_test), mse(preds, y_test)

In [11]:
eval_regression_model(lr, X_test, y_test)

(3.6187, 25.7671)

Works!

#### Testing momentum

In [12]:
from lincoln.optimizers import SGDMomentum

In [18]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())
trainer = Trainer(lr, SGDMomentum(momentum=0.1))

trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);
eval_regression_model(lr, X_test, y_test)

Validation loss after 1 epochs is 5581.277
Validation loss after 2 epochs is 4204.301
Validation loss after 3 epochs is 5624.175
Validation loss after 4 epochs is 4249.497
Validation loss after 5 epochs is 3933.467
Validation loss after 6 epochs is 4563.716
Validation loss after 7 epochs is 3766.584
Validation loss after 8 epochs is 4485.246
Validation loss after 9 epochs is 4700.247
Validation loss after 10 epochs is 3917.355


(3.6283, 25.7721)

#### "Neural Network" Regression

In [19]:
nn = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer = Trainer(nn, SGDMomentum(momentum=0.1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 5187.280
Validation loss after 2 epochs is 4834.083
Validation loss after 3 epochs is 3436.267
Validation loss after 4 epochs is 2984.902
Validation loss after 5 epochs is 2956.575
Validation loss after 6 epochs is 2364.621
Validation loss after 7 epochs is 2382.822
Validation loss after 8 epochs is 2282.458
Validation loss after 9 epochs is 2447.009
Validation loss after 10 epochs is 2119.687


In [20]:
eval_regression_model(nn, X_test, y_test)

(2.3471, 13.9453)

Works!

#### "Deep Learning" Regression

In [21]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer = Trainer(dl, SGDMomentum(momentum=0.1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 10378.049
Validation loss after 2 epochs is 5909.530
Validation loss after 3 epochs is 4264.764
Validation loss after 4 epochs is 3561.067
Validation loss after 5 epochs is 2714.385
Validation loss after 6 epochs is 3809.022
Validation loss after 7 epochs is 3245.006
Validation loss after 8 epochs is 4771.190
Validation loss after 9 epochs is 3646.437
Validation loss after 10 epochs is 4917.084


In [22]:
eval_regression_model(dl, X_test, y_test)

(4.5132, 32.3492)

In [23]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError())

trainer2 = Trainer(dl, SGDMomentum())
trainer2.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 9048.820
Validation loss after 2 epochs is 9242.333
Validation loss after 3 epochs is 9876.866
Validation loss after 4 epochs is 9357.191
Validation loss after 5 epochs is 8151.121
Validation loss after 6 epochs is 7768.742
Validation loss after 7 epochs is 7168.652
Validation loss after 8 epochs is 6943.183
Validation loss after 9 epochs is 11603.037
Validation loss after 10 epochs is 8359.214


In [24]:
eval_regression_model(dl, X_test, y_test)

(5.0506, 54.9948)

Works!

## Classification models

#### Data prep (SciKit Learn)

In [25]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [26]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

#### "`LogSigmoid`" Logistic Regression

In [27]:
logr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LogSigmoid())],
            loss = LogSigmoidLoss())

trainer = Trainer(logr, SGD())
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
       single_output=True);

Validation loss after 10 epochs is 23.003
Validation loss after 20 epochs is 19.235
Validation loss after 30 epochs is 18.084
Validation loss after 40 epochs is 17.586
Validation loss after 50 epochs is 17.340
Validation loss after 60 epochs is 17.210
Validation loss after 70 epochs is 17.108
Validation loss after 80 epochs is 17.061
Validation loss after 90 epochs is 17.031
Validation loss after 100 epochs is 17.019


In [28]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError())

trainer = Trainer(logr_mse, SGD())
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
       single_output=True);

Validation loss after 10 epochs is 10.025
Validation loss after 20 epochs is 7.280
Validation loss after 30 epochs is 6.160
Validation loss after 40 epochs is 5.527
Validation loss after 50 epochs is 5.118
Validation loss after 60 epochs is 4.818
Validation loss after 70 epochs is 4.576
Validation loss after 80 epochs is 4.395
Validation loss after 90 epochs is 4.274
Validation loss after 100 epochs is 4.190


In [29]:
def accuracy(y_true, y_pred):
    return round(torch.sum(torch.eq(y_true, y_pred)).item() / y_true.size()[0], 4)

In [30]:
def eval_classification_model(model: NeuralNetwork, 
                              X_test: Tensor, 
                              y_test: Tensor,
                              log_probs: bool = False):
    preds = model.forward(X_test)
    if log_probs:
        preds = torch.exp(preds)
    preds = preds > 0.5
    preds = preds.reshape(preds.shape[0]).type(torch.FloatTensor)  
    return accuracy(preds, y_test)

In [31]:
eval_classification_model(logr, X_test, y_test, log_probs=True)

0.9766

In [32]:
eval_classification_model(logr_mse, X_test, y_test)

0.9708

`LogSoftmaxLoss` works!

In [33]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError())
trainer_m = Trainer(logr_mse, SGDMomentum())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
       single_output=True);

Validation loss after 10 epochs is 4.649
Validation loss after 20 epochs is 3.897
Validation loss after 30 epochs is 3.723
Validation loss after 40 epochs is 3.613
Validation loss after 50 epochs is 3.615
Validation loss after 60 epochs is 3.736
Validation loss after 70 epochs is 3.778
Validation loss after 80 epochs is 3.822
Validation loss after 90 epochs is 3.893
Validation loss after 100 epochs is 3.965


In [34]:
eval_classification_model(logr_mse, X_test, y_test)

0.9708

#### "Neural Network" Logistic Regression

In [35]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError())
trainer_m = Trainer(logr_nn, SGDMomentum())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);

Validation loss after 10 epochs is 5.820
Validation loss after 20 epochs is 4.857
Validation loss after 30 epochs is 4.324
Validation loss after 40 epochs is 4.022
Validation loss after 50 epochs is 3.850
Validation loss after 60 epochs is 3.736
Validation loss after 70 epochs is 3.679
Validation loss after 80 epochs is 3.735
Validation loss after 90 epochs is 3.721
Validation loss after 100 epochs is 3.895


In [36]:
eval_classification_model(logr_nn, X_test, y_test)

0.9708

**Works!**

In [ ]:
def _conv_transform(input_: Tensor) -> Tensor:

    return input_.view(input_.shape[0],
                       input_.shape[1] * input_.shape[2] * input_.shape[3])

X_train_flat = _conv_transform(X_train)
X_test_flat = _conv_transform(X_test)

In [ ]:
conv_nn_ll = NeuralNetwork(
    layers=[Dense(neurons=250, 
                  activation=Sigmoid()),
            Dense(neurons=50, 
                  activation=Sigmoid()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_ll.fit(X_train_flat, y_train, X_test_flat, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

In [ ]:
def eval_nn(nn, X_test, y_test):
    test_preds = nn.forward(X_test)
    
    test_preds_l = torch.argmax(test_preds, dim=1)
    
    test_vals_l = torch.argmax(y_test, dim=1)
    
    return torch.sum(test_preds_l == test_vals_l)

In [ ]:
eval_nn(conv_nn_ll, X_test_flat, y_test)

In [ ]:
X_train_perm = X_train.permute(0, 3, 1, 2)
X_test_perm = X_test.permute(0, 3, 1, 2)

In [ ]:
# settings
pytorch = True

In [ ]:
conv_nn_logloss = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_logloss.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

In [ ]:
eval_nn(conv_nn_logloss, X_test_perm, y_test)

### Testing network without PyTorch layers 

In [ ]:
# %load_ext autoreload
# %autoreload 2

#### No Cython

In [ ]:
pytorch = False
cython = False

In [ ]:
conv_nn_normal = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_normal.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

#### Cython

In [ ]:
pytorch = False
cython = True

In [ ]:
conv_nn_cython = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_cython.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")